In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup 
import pandas as pd

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
dataframe = []
driver.get('https://www.goodreads.com/choiceawards/best-books-2021')
page_source = BeautifulSoup(driver.page_source)
Categories = page_source.find('div', id = 'categories')
CategoriesURLs = []
for x in Categories.select('a'):
    if x['href'] != '#':
        CategoriesURL = 'https://www.goodreads.com'+x['href']
        CategoriesURLs.append(CategoriesURL)
CategoriesURLs

In [ ]:
def scraping(webulr):
    driver.get(webulr)
    page_source1 = BeautifulSoup(driver.page_source)
    Votes = []
    Titles = []
    BookULRs = []
    for item in page_source1.select('strong', class_ ='uitext result'):
        Vote = item.get_text().strip()
        Votes.append(Vote)
    for item in page_source1.find_all('a', class_ ='pollAnswer__bookLink'):
        title = item.find('img')['alt']
        Titles.append(title)   
        BookULR = 'https://www.goodreads.com/'+item['href'] 
        BookULRs.append(BookULR)
    five_star = [] 
    four_star = []
    three_star = []
    two_star = []
    one_star = []
    ratingCount = []
    reviewCount= []
    for item in BookULRs:
        driver.get(item)
        driver.find_element_by_id('rating_details').click()
        page_source2 = BeautifulSoup(driver.page_source)
        five_star.append(page_source2.find('div', class_='tooltip goodreads').find_all('td', width="90")[0].get_text().strip())
        four_star.append(page_source2.find('div', class_='tooltip goodreads').find_all('td', width="90")[1].get_text().strip())
        three_star.append(page_source2.find('div', class_='tooltip goodreads').find_all('td', width="90")[2].get_text().strip())
        two_star.append(page_source2.find('div', class_='tooltip goodreads').find_all('td', width="90")[3].get_text().strip())
        one_star.append(page_source2.find('div', class_='tooltip goodreads').find_all('td', width="90")[4].get_text().strip())
        ratingCount.append(page_source2.find('meta', itemprop='ratingCount')['content'].strip())
        reviewCount.append(page_source2.find('meta', itemprop='reviewCount')['content'].strip())
    Listbooks = {'Title': Titles, 'Vote': Votes, 'Five_star': five_star, 'Four_star': four_star, 'Three_star': three_star, 'Two_star': two_star, 'One_star': one_star, 'RatingCount': ratingCount
    ,'ReviewCount': reviewCount}
    df = pd.DataFrame(Listbooks)
    df['Total Vote']=page_source1.find('div', class_="greyText gcaNumVotes").get_text().strip()
    df['Categorie']= page_source1.find('div', class_="gcaMastheader u-marginLeftMedium").get_text().strip()
    return df

dataframe = pd.DataFrame()
for item in CategoriesURLs[15:]:
    x = scraping(item)
    dataframe = dataframe.append(x)
dataframe.to_excel(r'D:\Project\220411_Goodreads\book3.xlsx', index=False, encoding='utf-8')